In [2]:
import numpy as np
import pandas as pd
import os
import sys
import tensorflow as tf
import xgboost as xgb
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection  import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost.sklearn import XGBRegressor
import LightGBM as lgb
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso

In [47]:
train_set = pd.read_csv('./data/train_set.csv',encoding='gbk')

/Users/lmlyz/Library/Python/3.6/lib/python/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (89,92,93,94,95,96,100,101,102,103,105,122,124,125,129,130,131,132,133,134,135,136,137,144,145,149,150,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,178,182,183,193,198,199,210,214,216,217,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
df_part2 = pd.read_csv('./data/part2.csv',sep='$')

In [48]:
test_set = pd.read_csv('./data/test_set.csv',encoding='gbk')

/Users/lmlyz/Library/Python/3.6/lib/python/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (92,93,94,95,96,101,103,104,105,122,124,125,129,130,131,133,134,135,136,137,144,145,150,152,162,182,193,214,216,217,218,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
part2_col = list(set(df_part2['table_id']))

In [39]:
print(train_set.shape,test_set.shape)

(38199, 245) (9538, 245)


In [13]:
def resetIndex(df):
    vid = df['vid']
    df = df.drop('vid',axis=1)
    df = df.set_index(vid)
    return df

In [9]:
col = ['收缩压','舒张压','血清甘油三酯','血清高密度脂蛋白','血清低密度脂蛋白']

In [14]:
### 过滤掉part1的数据 保留需要预测的数据
def filter_col(df,col,part2_col):
    
    for feature in df.columns:
        if feature not in part2_col:
            if feature not in col:
                df = df.drop(feature,axis=1)
        
    return df
    

In [62]:
train_set = filter_col(train_set,col,part2_col)

In [63]:
test_set = filter_col(test_set,col,part2_col)

In [178]:
train_set.head()

,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,100005,100006,100007,100012,...,979014,979015,979016,979017,979018,979019,979020,979021,979022,979023
0,002d1e4859fafd9ded2a2e1e7c839b62,165.0,100.0,2.08,1.29,3.24,12.9,NaN,0.26,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,92dd479df5e30ab6a0a1cf85ac53efc3,141.0,97.0,2.64,1.36,4.75,13.0,NaN,0.16,161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6bb59d517c4c70f8f50844d24fbd0355,120.0,80.0,1.37,1.25,2.66,NaN,13.8,0.26,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0ebb42adae512906f7e1135da734ea63,100.0,70.0,1.27,2.21,1.73,NaN,13.4,0.26,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ebe7811e919109c42c092abbd98b4ca6,110.0,80.0,0.80,1.87,2.21,NaN,11.9,0.20,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
test_set.shape

(9538, 132)

In [66]:
#train_set.to_csv('./temp/train_set_p2.csv',encoding='gbk')

In [67]:
#test_set.to_csv('./temp/test_set_p2.csv',encoding='gbk')

In [72]:
x = '44.0 44.0'

In [73]:
str(x).isdigit()

False

In [3]:
###重新读取数train_set_p2
train_set = pd.read_csv('./temp/train_set_p2.csv',encoding='gbk')

/Users/lmlyz/Library/Python/3.6/lib/python/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7,9,10,11,12,13,14,15,16,19,22,27,29,30,31,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,65,66,72,73,74,78,84,88,89,90,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#len(train_set.columns)
train_set.head()

,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,100005,100006,100007,100012,...,979014,979015,979016,979017,979018,979019,979020,979021,979022,979023
0,002d1e4859fafd9ded2a2e1e7c839b62,165.0,100.0,2.08,1.29,3.24,12.9,NaN,0.26,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,92dd479df5e30ab6a0a1cf85ac53efc3,141.0,97.0,2.64,1.36,4.75,13.0,NaN,0.16,161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6bb59d517c4c70f8f50844d24fbd0355,120.0,80.0,1.37,1.25,2.66,NaN,13.8,0.26,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0ebb42adae512906f7e1135da734ea63,100.0,70.0,1.27,2.21,1.73,NaN,13.4,0.26,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ebe7811e919109c42c092abbd98b4ca6,110.0,80.0,0.80,1.87,2.21,NaN,11.9,0.20,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
test_set = pd.read_csv('./temp/test_set_p2.csv',encoding='gbk')

/Users/lmlyz/Library/Python/3.6/lib/python/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10,11,12,13,14,16,18,19,20,22,23,24,28,29,30,32,33,34,35,36,42,43,44,54,74,85,104,106,107,108,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
test_set.head()

,Unnamed: 0,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,100005,100006,100007,100010,...,979014,979015,979016,979017,979018,979019,979020,979021,979022,979023
0,0,NaN,NaN,NaN,NaN,NaN,11.5,NaN,0.200,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,12.4,0.230,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- 0CELL/uL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,43.7,11.2,0.360,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,41.1,13.0,0.291,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
test_set = test_set.drop(col,axis=1)

In [11]:
test_set.head()

,Unnamed: 0,100005,100006,100007,100010,100012,100014,10002,10003,10004,...,979014,979015,979016,979017,979018,979019,979020,979021,979022,979023
0,0,11.5,NaN,0.200,NaN,148,15.87,11.4,NaN,4.74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,12.4,0.230,-,NaN,NaN,NaN,NaN,4.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,- 0CELL/uL,NaN,NaN,NaN,NaN,3.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,43.7,11.2,0.360,-,NaN,NaN,5.6,28.23,4.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,41.1,13.0,0.291,-,NaN,NaN,NaN,NaN,6.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
train_set = resetIndex(train_set)

In [16]:
def drop_col(test_set,train_set):
    for col in test_set.columns:
        if col not in train_set.columns:
            test_set = test_set.drop(col,axis=1)
    print(test_set.shape)
    return test_set

In [17]:
test_set = drop_col(test_set,train_set)

(9538, 104)


In [18]:
test_set.head()

,100005,100006,100007,100012,100014,10002,10003,10004,1106,1107,...,979014,979015,979016,979017,979018,979019,979020,979021,979022,979023
0,11.5,NaN,0.200,148,15.87,11.4,NaN,4.74,1.29,0.81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,12.4,0.230,NaN,NaN,NaN,NaN,4.34,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.95,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,43.7,11.2,0.360,NaN,NaN,5.6,28.23,4.02,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41.1,13.0,0.291,NaN,NaN,NaN,NaN,6.99,1.66,0.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
idx = pd.read_csv('./submit/submit11.csv',encoding='gbk',header=None)

In [21]:
test_set['vid'] = idx[0]

In [22]:
test_set = resetIndex(test_set)

In [23]:
test_set = drop_col(test_set,train_set)

(9538, 104)


In [186]:
#train_set.to_csv('./temp/train_set_p2.csv',encoding='gbk')

In [24]:
def clean_label(x):
    x = str(x)
    
  
    if len(x.split(sep='.'))>2:#2.2.8
        i = x.rindex('.')
        x = x[0:i]+x[i+1:]
    
    if '。' in x:
        x = x.replace('。','.')
    if ', ' in x:
        x = x.replace(', ','.')
    if x == ' ' or x == '()':
        x = np.nan
    if '标本已退' in str(x) or '未做' in str(x) or '未查' in str(x) or '弃查' in str(x):
        x = np.nan
    if str(x).isdigit()==False and len(str(x))>4:
        x = x[0:4]
    if 'A' in str(x):
        x = np.nan
        
    if 'CLT' in str(x):
        x = np.nan
        
      
    return x
    

In [46]:
train_set_tmp1 = train_set.applymap(clean_label)

In [47]:
test_set1_tmp1 = test_set.applymap(clean_label)

In [313]:
#train_set = pd.read_csv('./temp/train_set_p2_cleaned.csv',encoding='gbk')

/Users/lmlyz/Library/Python/3.6/lib/python/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (13,26,27,32,36,48,63,65,67,72,73,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [97]:
train_set_tmp1 = pd.read_csv('./temp/train_set_p2_cleaned.csv',encoding='gbk')
train_set_tmp1 = resetIndex(train_set_tmp1)

/Users/lmlyz/Library/Python/3.6/lib/python/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7,11,17,18,25,32,33,34,38,39,40,48,65,74,88,89,90) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [50]:
test_set1_tmp1.to_csv('./temp/test_set_p2_cleaned.csv',encoding='gbk')

In [31]:
def drop_row(df,thresh = 0.35):
    num_rows = len(df.index)
    num_cols = len(df.columns)
    rows = []
    for i in range(num_rows):
        if float(df.iloc[i].count()) / num_cols < thresh:
            rows.append(df.index[i])
            
    print(len(rows))
    df = df.drop(rows,axis=0)
    
    return df

In [133]:
import re
def func1(x):
    regexp = re.compile(r'[^\x00-\x7f]')
    x = str(x)
    if(regexp.search(x)):
        x = np.nan
    if '----' in str(x):
        x = np.nan
    if '....' == str(x):
        x = np.nan
    if str(x) == '':
        x = np.nan
    if str(x) == '.':
        x = np.nan
    if '.()' in str(x):
        x = np.nan
    return x

In [33]:
def func2(x):
    x = str(x)
    if '>' in x:#> 11.00
        i = x.index('>')
        x = x[i+1:]
    if '<' in x:
        i = x.index('<')
        x = x[i+1:]
    if '=' in x:
        i = x.index('=') # =1.
        x = x[i+1:]
    if ' ' in x:
        x = x.replace(' ','.') # 50 5
    if '-' in x or '+' in x:
        x = np.nan
    return x

In [51]:
#train_set_tmp1 = train_set_tmp1.applymap(lambda x:float(x))
#drop_row(train_set)
#train_set = train_set.applymap(func1)
#train_set = train_set.astype('float64')

In [81]:
train_set_tmp11 = train_set_tmp1

In [134]:
train_set_tmp11 = train_set_tmp11.applymap(func1)

In [104]:
train_set_tmp11 = train_set_tmp11.applymap(func2)

In [98]:
train_set_tmp11 = pd.read_csv('./temp/train_set_tmp11.csv',encoding='gbk')
train_set_tmp11 = resetIndex(train_set_tmp11)

/Users/lmlyz/Library/Python/3.6/lib/python/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7,26,38,39,40,65,74,88,89,90) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [99]:
train_set_tmp11 = train_set_tmp11.replace('....',str(np.nan))

In [100]:
train_set_tmp11 = train_set_tmp11.replace('...',str(np.nan))

In [101]:
train_set_tmp11 = train_set_tmp11.applymap(clean_label)

In [109]:
train_set_tmp11 = train_set_tmp11.replace('.','.')

In [139]:
train_set_tmp11 = train_set_tmp11.applymap(lambda x: float(x))

In [140]:
train_set_tmp11.to_csv('./temp/train_set_tmp12.csv',encoding='gbk')

In [379]:
test_set = test_set.applymap(func2)
#test_set.to_csv('./temp/test_set_p2_cleaned.csv',encoding='gbk')

In [150]:
test_set1_tmp11 = test_set1_tmp1       

In [151]:
test_set1_tmp11 = test_set1_tmp11.applymap(func1)

In [152]:
test_set1_tmp11 = test_set1_tmp11.applymap(func2)

In [154]:
test_set1_tmp11 = test_set1_tmp11.applymap(clean_label)

In [155]:
test_set1_tmp11 = test_set1_tmp11.applymap(lambda x:float(x))

In [157]:
test_set1_tmp11.to_csv('./temp/test_set_tmp12.csv',encoding='gbk')

In [383]:
#test_set.to_csv('./temp/test_set_p2_cleaned.csv',encoding='gbk')

In [416]:
t = train_set

In [169]:
train_set11 = drop_row(train_set_tmp11,0.3)

7745


In [171]:
train_set12 = drop_row(train_set,0.35)

15683


In [172]:
train_set11.shape

(30454, 109)

In [173]:
train_set12.shape

(22516, 109)

In [176]:
train_set11.to_csv('./temp/train_set11_droped.csv',encoding='gbk')

In [177]:
train_set12.to_csv('./temp/train_set12_droped.csv',encoding='gbk')

In [178]:
def drop_col_thresh(df,thresh = 0.25):
    
    num_rows = len(df.index)
    cols = []
    for item in df.columns:
        if float(df[item].count()) / num_rows < thresh:
            cols.append(item)
            
    print(len(cols))
    
    df = df.drop(cols,axis=1)
    #pass
    return df

In [179]:
train_set111 = drop_col_thresh(train_set11)

43


In [180]:
train_set121 = drop_col_thresh(train_set12)

19


In [181]:
train_set111.describe()

,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,100005,100006,100007,10002,10003,...,317,319,3193,32,320,33,34,37,38,39
count,30447.000000,30447.000000,30454.000000,30454.000000,30454.000000,11831.000000,15054.000000,13424.000000,17302.000000,15634.000000,...,15229.000000,15290.000000,29449.000000,15137.000000,14865.000000,14993.000000,12492.000000,15290.000000,15290.000000,12636.000000
mean,125.785135,76.855618,1.621095,1.403891,2.758533,19.178379,14.623996,0.743764,9.145357,28.654682,...,333.977149,213.796599,1.018061,3.768095,9.416341,2.068368,0.429653,57.851655,30.766602,7.787274
std,19.221041,13.589074,1.338751,0.335375,0.845126,12.216086,2.585549,4.429144,4.037982,4.170400,...,14.953268,55.857126,0.006649,1.257112,1.820593,0.734522,0.209281,13.175705,10.681922,6.355051
min,0.000000,0.000000,0.100000,0.150000,0.000000,4.000000,0.000000,0.000000,0.140000,7.200000,...,248.000000,28.000000,1.000000,0.430000,0.000000,0.200000,0.090000,0.310000,0.120000,0.010000
25%,112.000000,68.000000,0.890000,1.160000,2.170000,12.100000,12.600000,0.160000,6.370000,25.800000,...,325.000000,176.000000,1.010000,2.900000,8.300000,1.600000,0.300000,53.700000,26.500000,4.900000
50%,123.000000,76.000000,1.280000,1.350000,2.680000,13.000000,15.300000,0.200000,8.500000,28.500000,...,333.000000,211.000000,1.020000,3.600000,9.500000,2.000000,0.400000,59.500000,32.000000,6.700000
75%,137.000000,85.000000,1.910000,1.600000,3.250000,15.100000,16.200000,0.260000,11.100000,31.300000,...,343.000000,248.000000,1.020000,4.400000,10.600000,2.400000,0.500000,65.000000,37.400000,8.700000
max,252.000000,1001.000000,28.800000,4.780000,11.500000,56.800000,31.800000,51.800000,52.100000,79.700000,...,485.000000,610.000000,1.030000,18.300000,20.100000,14.600000,4.100000,90.200000,84.800000,55.800000


In [184]:
def drop_with_mean(df):
    rows = []
    col_mean = {}
    for item in df.columns:
        col_mean[item] = df[item].mean()
    
    for i in range(len(df.index)):
        for k,v in df.iloc[i].items():
            if np.isnan(v) == False:
                if v > 3.0*col_mean[k]:
                    rows.append(df.index[i])
    print(len(rows))
    
    df = df.drop(rows,axis=0)
    
    return df

In [185]:
t = train_set111

In [186]:
t = drop_with_mean(t)


6212


In [187]:
t.to_csv('./temp/train_set111_drop_mean.csv',encoding='gbk')

In [481]:
train_set1 = resetIndex(train_set1)

In [204]:
t.columns[17+1+5]

'192'

In [199]:
data = t

In [222]:
tt = data[data['100005'].notnull()]

In [223]:
tt

,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,100005,100006,100007,10002,10003,...,317,319,3193,32,320,33,34,37,38,39
vid,,,,,,,,,,,,,,,,,,,,,
002d1e4859fafd9ded2a2e1e7c839b62,165.0,100.0,2.08,1.29,3.24,12.9,NaN,0.26,6.30,29.1,...,330.0,255.0,1.01,4.10,10.5,2.0,0.20,65.6,31.8,2.6
086d3fa10ae104f6d27b1e37d8462398,132.0,73.0,0.84,1.71,2.87,46.3,11.1,0.24,NaN,NaN,...,348.0,270.0,1.01,3.80,9.8,1.5,0.40,66.9,26.1,7.0
917f4674052241ce1b33bc8429f3263b,155.0,84.0,1.85,1.12,4.23,37.2,13.3,0.37,NaN,NaN,...,341.0,145.0,1.02,3.50,11.4,2.2,0.40,57.0,36.0,8.0
53e02ec25395372d4e3d07f63689d7a6,127.0,80.0,0.69,1.67,2.54,42.7,10.9,0.24,NaN,NaN,...,336.0,320.0,1.01,4.30,10.0,1.6,0.50,66.9,25.3,7.8
0a238167c1c9778ed3cb6867b5407f3b,136.0,94.0,1.14,1.29,3.76,39.5,12.9,0.29,16.50,25.3,...,350.0,190.0,1.02,3.40,10.6,1.3,0.50,64.9,24.8,10.3
3cd05fc4212d5fa0fee7909fdc39119e,98.0,54.0,1.18,1.46,3.10,44.1,13.5,0.34,9.04,24.3,...,327.0,274.0,1.02,5.20,11.2,3.8,0.90,52.0,39.0,9.0
771f6689382f9ae933bb135c9aab700f,129.0,70.0,0.36,2.12,2.15,45.4,16.3,0.15,5.64,29.9,...,326.0,232.0,1.02,5.70,11.3,1.7,0.70,70.0,21.0,9.0
bbba4e711b50d7a650844a7e14ec8a0a,136.0,71.0,2.40,1.25,3.70,41.3,14.4,0.24,NaN,NaN,...,348.0,117.0,1.03,3.60,9.4,1.4,0.30,68.9,25.8,5.3
b778070e13885bc3a2de2c5c7b00ec95,91.0,61.0,1.45,1.19,3.46,45.9,14.0,0.34,NaN,NaN,...,336.0,231.0,1.02,3.50,11.1,2.0,0.40,59.0,33.0,8.0


In [487]:
### 重新读取数据
train_set1 = pd.read_csv('./temp/train_set1_drop_mean.csv',encoding='gbk')

In [488]:
train_set2 = pd.read_csv('./temp/train_set2_drop_mean.csv',encoding='gbk')

In [489]:
train_set1.head()

,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,100005,100006,100007,10002,...,317,319,3193,32,320,33,34,37,38,39
0,002d,165.0,100.0,2.08,1.29,3.24,12.9,NaN,0.26,6.3,...,330.0,255.0,1.01,4.1,10.5,2.0,0.2,65.6,31.8,2.6
1,0ebb,100.0,70.0,1.27,2.21,1.73,NaN,13.4,0.26,16.8,...,354.0,152.0,1.02,1.9,10.0,2.0,0.1,45.4,51.0,3.6
2,c269,110.0,80.0,1.40,1.74,2.09,NaN,9.9,0.13,NaN,...,350.0,319.0,1.02,2.5,8.2,2.0,0.7,48.4,37.7,13.9
3,d701,130.0,78.0,2.50,1.22,4.53,NaN,NaN,NaN,NaN,...,NaN,NaN,1.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,c004,164.0,98.0,1.12,1.51,2.79,NaN,NaN,NaN,NaN,...,NaN,NaN,1.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [225]:
test_set1_tmp11.head()

,100005,100006,100007,100012,100014,10002,10003,10004,1106,1107,...,979014,979015,979016,979017,979018,979019,979020,979021,979022,979023
vid,,,,,,,,,,,,,,,,,,,,,
c643a744e2e94f3ff354d920958bd37b,11.5,NaN,0.20,148.0,15.8,11.4,NaN,4.74,1.29,0.81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a81c1fc0cfa8c0fbc450d2ffeae14b9f,NaN,12.4,0.23,NaN,NaN,NaN,NaN,4.34,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2af0da8f6f0b8d642f88b3a8194408b6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.95,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2bd361a4c5b9b0d5aab62c101daa4ee6,43.7,11.2,0.36,NaN,NaN,5.6,28.2,4.02,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16107e514ed0f51c4e39b1a0fa5a04f9,41.1,13.0,0.29,NaN,NaN,NaN,NaN,6.99,1.66,0.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [491]:
train_set1 = resetIndex(train_set1)
train_set2 = resetIndex(train_set2)

In [519]:
train_set1 = train_set1.fillna(train_set1.mean())

In [520]:
train_set2 = train_set2.fillna(train_set2.mean())

In [522]:
train_set1.to_csv('./temp/train_set1_fillna.csv',encoding='gbk')

In [523]:
train_set2.to_csv('./temp/train_set2_fillna.csv',encoding='gbk')

In [226]:
#104
test_set1 = drop_col(test_set1_tmp11,train_set111)

(9538, 61)


In [228]:
test_set1.to_csv('./temp/test_set111.csv',encoding='gbk')

In [526]:
test_set2 = drop_col(test_set,train_set2)

(9538, 85)


In [528]:
test_set1 = test_set1.fillna(test_set1.mean())

In [529]:
test_set2 = test_set2.fillna(test_set2.mean())

In [530]:
test_set1.to_csv('./temp/test_set1_fillna.csv',encoding='gbk')

In [531]:
test_set2.to_csv('./temp/test_set2_fillna.csv',encoding='gbk')

In [575]:
def norm(df,col,flag = True):
    scalar = StandardScaler()
    if flag:
        y = df[col]
        t = df.drop(col,axis=1)
        scalar.fit(t)
        df2 = scalar.transform(t)
        X = pd.DataFrame(data=df2,index=df.index,columns = t.columns)
        X[col]= y
    else:
        t = df
        scalar.fit(t)
        df2 = scalar.transform(t)
        X = pd.DataFrame(data=df2,index=df.index,columns = t.columns)
    return X

In [536]:
train_set1 = pd.read_csv('./temp/train_set1_fillna.csv',encoding='gbk')

In [556]:
train_set1 = resetIndex(train_set1)

In [563]:
t = train_set1

In [564]:
t = norm(t,col)

In [569]:
k = train_set2

In [570]:
k = norm(k,col)

In [571]:
k.head()

,100005,100006,100007,10002,10003,10004,1115,1117,1127,1345,...,979019,979020,979021,979022,979023,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
vid,,,,,,,,,,,,,,,,,,,,,
002d,-7.987239e-01,9.493109e-15,9.010224e-01,-8.709096e-01,1.081482e-01,-1.546583,-8.173707e-02,-2.954810e-01,-2.158872e-15,1.755981e-15,...,1.245730e-15,3.565445e-15,-4.465087e-15,-1.882247e-15,-5.921423e-15,165.0,100.0,2.08,1.29,3.24
0ebb,-4.673270e-15,-7.817852e-01,9.010224e-01,2.509134e+00,-1.701789e+00,0.069529,-7.171043e-01,-4.173702e-01,1.234534e+00,1.891526e+00,...,1.245730e-15,3.565445e-15,-4.465087e-15,-1.882247e-15,-5.921423e-15,100.0,70.0,1.27,2.21,1.73
d701,-4.673270e-15,9.493109e-15,6.475203e-15,-6.290076e-15,1.634794e-14,-0.924380,3.762130e-15,-6.495559e-16,-2.158872e-15,1.755981e-15,...,1.245730e-15,3.565445e-15,-4.465087e-15,-1.882247e-15,-5.921423e-15,130.0,78.0,2.50,1.22,4.53
0a23,3.089039e+00,-1.078682e+00,1.439393e+00,2.412561e+00,-1.057574e+00,-1.150635,-5.741467e-01,-2.650087e-01,-2.158872e-15,2.324007e+00,...,1.245730e-15,3.565445e-15,-4.465087e-15,-1.882247e-15,-5.921423e-15,136.0,94.0,1.14,1.29,3.76
3cd0,3.761359e+00,-7.224058e-01,2.336676e+00,1.112085e-02,-1.364343e+00,0.457396,9.613242e-01,-6.794318e-01,-2.158872e-15,-7.651472e-01,...,1.245730e-15,3.565445e-15,-4.465087e-15,-1.882247e-15,-5.921423e-15,98.0,54.0,1.18,1.46,3.10


In [572]:
test_set1.head()

,100005,100006,100007,10002,10003,10004,1115,1117,1345,155,...,317,319,3193,32,320,33,34,37,38,39
vid,,,,,,,,,,,,,,,,,,,,,
c643a744e2e94f3ff354d920958bd37b,11.500000,14.465599,0.200000,11.400000,28.563335,4.74,142.000000,89.000000,169.000000,4.180000,...,284.000000,203.000000,1.018049,3.500000,10.000000,2.800000,0.500000,51.200000,41.200000,7.600000
a81c1fc0cfa8c0fbc450d2ffeae14b9f,19.044075,12.400000,0.230000,9.174319,28.563335,4.34,75.270523,38.000000,173.473871,2.334328,...,333.000000,244.000000,1.020000,4.700000,9.600000,2.000000,0.700000,62.900000,27.700000,9.400000
2af0da8f6f0b8d642f88b3a8194408b6,19.044075,14.465599,0.637103,9.174319,28.563335,3.95,75.270523,34.037286,173.473871,2.334328,...,334.134065,214.913396,1.010000,3.783944,9.417413,2.078969,0.427667,57.470735,30.651793,7.576224
2bd361a4c5b9b0d5aab62c101daa4ee6,43.700000,11.200000,0.360000,5.600000,28.200000,4.02,55.900000,79.400000,196.000000,2.334328,...,350.000000,188.000000,1.010000,3.900000,7.000000,3.100000,0.800000,49.300000,40.100000,10.600000
16107e514ed0f51c4e39b1a0fa5a04f9,41.100000,13.000000,0.290000,9.174319,28.563335,6.99,75.270523,34.037286,173.473871,2.334328,...,353.000000,180.000000,1.020000,3.300000,10.500000,1.800000,0.600000,58.000000,31.000000,11.000000


In [573]:
ts1 = test_set1

In [576]:
ts1 = norm(ts1,col,0)

In [578]:
ts2 = test_set2

In [579]:
ts2 = norm(ts2,col,0)

In [580]:
ts1.to_csv('./temp/test_set1_normed.csv',encoding='gbk')

In [581]:
ts2.to_csv('./temp/test_set2_normed.csv',encoding='gbk')

In [582]:
t.shape

(20954, 66)

In [583]:
k.shape

(13277, 90)

In [585]:
t.to_csv('./temp/train_set1_normed.csv',encoding='gbk')

In [586]:
k.to_csv('./temp/train_set2_normed.csv',encoding='gbk')